In [24]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import sqlite3  # For SQLite database

# Site URL
url = "https://merolagani.com/LatestMarket.aspx"

# Make a GET request to fetch the raw HTML data
con = requests.get(url)

# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(con.text, 'html.parser')

# Find the table containing the market data
table = soup.find('table', class_='table table-hover live-trading sortable')

# Get the headers from the table (e.g., Symbol, LTP, etc.)
headers = [header.text.strip() for header in table.find_all('th')]

# Extract the rows (tr) of the table
rows = table.find_all('tr', class_=["increase-row", "decrease-row", "nochange-row"])

# Parse the data inside each row and store it in a dictionary format
result = []
for row in rows:
    cols = row.find_all('td')
    data = {
        headers[i]: cols[i].text.strip() if i < len(cols) else '' 
        for i in range(len(headers))
    }
    result.append(data)

# Convert the result into a DataFrame
df = pd.DataFrame(result)

# Clean up column names (strip extra spaces, remove empty columns)
df.columns = df.columns.str.strip()  # Remove any extra spaces
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]  # Remove any unnamed columns

# Explicitly drop the empty column (if any)
df = df.loc[:, df.columns != '']

# Rename columns to match the database column names
df.rename(columns={
    '% Change': 'Percentage_Change',
    'Qty.': 'Qty',
    'Diff.': 'Diff',
}, inplace=True)

# Save the DataFrame to a CSV file
df.to_csv('ScrapedData.csv', index=False)

# Print all columns to inspect the exact names
print("DataFrame columns after cleanup:")
print(df.columns)

# Check for missing values in 'PClose' and 'Diff' (after inspecting the columns)
if 'PClose' in df.columns and 'Diff' in df.columns:
    print("Missing values in 'PClose' and 'Diff':")
    print(df[['PClose', 'Diff']].isnull().sum())
    
    # Convert 'PClose' and 'Diff' columns to numeric values, forcing errors to NaN
    df['PClose'] = pd.to_numeric(df['PClose'], errors='coerce')
    df['Diff'] = pd.to_numeric(df['Diff'], errors='coerce')

    # Fill missing values with 0 in 'PClose' and 'Diff' columns
    df['PClose'] = df['PClose'].fillna(0)
    df['Diff'] = df['Diff'].fillna(0)
else:
    print("Columns 'PClose' or 'Diff' are not found in the DataFrame.")

# Connect to SQLite database (it will create the database if it doesn't exist)
conn = sqlite3.connect('stock_data.db')

# Ensure the 'stock_data' table is created if it doesn't exist
create_table_query = """
CREATE TABLE IF NOT EXISTS stock_data (
    Symbol TEXT,
    LTP REAL,
    Percentage_Change REAL,
    Open REAL,
    High REAL,
    Low REAL,
    Qty INTEGER,
    PClose REAL,
    Diff REAL
);
"""
conn.execute(create_table_query)

# Insert data into the 'stock_data' table
df.to_sql('stock_data', conn, if_exists='append', index=False)

# Commit and close the connection
conn.commit()
conn.close()

print("Data inserted into the database successfully!")


DataFrame columns after cleanup:
Index(['Symbol', 'LTP', 'Percentage_Change', 'Open', 'High', 'Low', 'Qty',
       'PClose', 'Diff'],
      dtype='object')
Missing values in 'PClose' and 'Diff':
PClose    0
Diff      0
dtype: int64
Data inserted into the database successfully!


In [ ]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('stock_data.db')

# Read data into a DataFrame
df = pd.read_sql_query("SELECT * FROM stock_data", conn)

# Print the DataFrame
print(df.head())

# Close the connection


# Fill missing values with 0
df['PClose'].fillna(0, inplace=True)
df['Diff'].fillna(0, inplace=True)

# Alternatively, drop rows with missing values
# df.dropna(subset=['PClose', 'Diff'], inplace=True)

conn.close()


    Symbol       LTP  Percentage_Change      Open      High       Low     Qty  \
0   ACLBSL    1056.0               2.11    1062.0    1025.3    1025.3    1533   
1     ADBL     366.1              -0.11     369.5     363.1     369.5   32031   
2   ACLBSL  1,056.00               2.11  1,062.00  1,025.30  1,025.30   1,533   
3     ADBL     366.1              -0.11     369.5     363.1     369.5  32,031   
4  ADBLD83  1,115.00               1.99  1,115.00  1,115.00  1,115.00      50   

   PClose  Diff  
0  1050.0   6.0  
1   365.0   1.1  
2     0.0   0.0  
3     0.0   0.0  
4     0.0   0.0  
PClose    0
Diff      0
dtype: int64


D:\Users\newac\AppData\Local\Temp\ipykernel_20292\1067290831.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['PClose'].fillna(0, inplace=True)
D:\Users\newac\AppData\Local\Temp\ipykernel_20292\1067290831.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when

In [23]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('stock_data.db')

# Fetch data from 'stock_data' table
query = "SELECT * FROM stock_data"
df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Display the DataFrame
print(df.head(50))


     Symbol        LTP  Percentage_Change       Open       High        Low  \
0    ACLBSL     1056.0               2.11     1062.0     1025.3     1025.3   
1      ADBL      366.1              -0.11      369.5      363.1      369.5   
2    ACLBSL   1,056.00               2.11   1,062.00   1,025.30   1,025.30   
3      ADBL      366.1              -0.11      369.5      363.1      369.5   
4   ADBLD83   1,115.00               1.99   1,115.00   1,115.00   1,115.00   
5       AHL      684.0               0.59      684.0      673.2      679.0   
6      AHPC      274.1              -1.01      279.0      272.6      279.0   
7     AKJCL      212.0               0.24      214.5      210.2      211.5   
8      AKPL      255.3              -0.66      262.1      254.5      262.1   
9     ALBSL   1,069.00               1.04   1,091.00   1,058.00   1,075.00   
10    ALICL      763.0               1.09      765.0      750.0      765.0   
11     ANLB   4,800.00               0.27   4,873.20   4,716.00 